In [ ]:
import marimo as mo
import pandas as pd
from czitools.metadata_tools import czi_metadata as czimd
from czitools.utils import misc
from czitools.read_tools import read_tools as czird
import dask.array as da
from pathlib import Path
import os
import glob
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
file_browser = mo.ui.file_browser(multiple=False)

# Display the file browser
mo.vstack([file_browser])

In [ ]:
filepath = str(file_browser.path(0))
print(f"Filepath: {filepath}")

Filepath: None


In [ ]:
def hex_to_rgb(hex_color: str) -> tuple[int, int, int]:
    """
    Convert a hexadecimal color string to an RGB tuple.
    Args:
        hex_color (str): A string representing a color in hexadecimal format (e.g., "#RRGGBB").
    Returns:
        tuple[int, int, int]: A tuple containing the RGB values as floats in the range [0, 1].
    """

    hex_color = hex_color.lstrip("#")
    rgb = tuple(int(hex_color[i : i + 2], 16) / 255.0 for i in (0, 2, 4))

    return rgb

# return array with dimension order STCZYX(A)
array6d, mdata = czird.read_6darray(filepath, use_dask=False, chunk_zyx=True)

# show dask array structure
if isinstance(array6d, da.Array):
    print(array6d)
else:
    print("Shape:", array6d.shape, "dtype:", array6d.dtype)

# Get array dimensions
dims = array6d.shape[:-2]
dims_names = ["S", "T", "C", "Z"]

cmaps = []

for ch in range(mdata.image.SizeC):
    chname = mdata.channelinfo.names[ch]
    rgb = hex_to_rgb(mdata.channelinfo.colors[ch][3:])
    cmaps.append(LinearSegmentedColormap.from_list(chname, [(0, 0, 0), rgb]))

<span class="codehilite"><div class="highlight"><pre><span></span><span class="gt">Traceback (most recent call last):</span>
  File <span class="nb">&quot;F:\Documents\anaconda3\envs\zen\Lib\site-packages\marimo\_runtime\executor.py&quot;</span>, line <span class="m">141</span>, in <span class="n">execute_cell</span>
<span class="w">    </span><span class="n">exec</span><span class="p">(</span><span class="n">cell</span><span class="o">.</span><span class="n">body</span><span class="p">,</span> <span class="n">glbls</span><span class="p">)</span>
  File <span class="nb">&quot;F:\AppData_TEMP\Temp\marimo_25684\__marimo__cell_bkHC_.py&quot;</span>, line <span class="m">16</span>, in <span class="n">&lt;module&gt;</span>
<span class="w">    </span><span class="n">array6d</span><span class="p">,</span> <span class="n">mdata</span> <span class="o">=</span> <span class="n">czird</span><span class="o">.</span><span class="n">read_6darray</span><span class="p">(</span><span class="n">filepath<

In [ ]:
scene = mo.ui.slider(
    start=0,
    stop=dims[0] - 1,
    step=1,
    label=f"scene [0 - {dims[0]-1}]",
    show_value=True,
)

time = mo.ui.slider(
    start=0,
    stop=dims[1] - 1,
    step=1,
    label=f"time [0 - {dims[1]-1}]",
    show_value=True,
)

channel = mo.ui.slider(
    start=0,
    stop=dims[2] - 1,
    step=1,
    label=f"channel [0 - {dims[2]-1}]",
    show_value=True,
)

zplane = mo.ui.slider(
    start=0,
    stop=dims[3] - 1,
    step=1,
    label=f"zplane [0 - {dims[3]-1}]",
    show_value=True,
)

mo.vstack([scene, time, channel, zplane])

In [ ]:
show_2dplane(scene.value, time.value, channel.value, zplane.value)

In [ ]:
def show_2dplane(s, t, c, z):
    plt.figure(figsize=(8, 8))
    plt.imshow(array6d[s, t, c, z, ...], cmap=cmaps[c], vmin=None, vmax=None)
    plt.tight_layout()
    return plt.gca()